# Modelado de texto

## Preprocesamiento de Texto

### Normalizar Texto

In [1]:
from nltk.corpus import stopwords as list_stopwords
import unicodedata
import re

PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|@—"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    minusculas=True,
                    punct=True,
                    accents=True,
                    num=True,
                    menciones=True,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """

    if minusculas:
        input_str = input_str.lower()

    if menciones:
        # Expresión regular para encontrar @ seguido de caracteres de palabra (\w+)
        input_str = re.sub(r'@\w+', '', input_str, flags=re.IGNORECASE)
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0

    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue

        if punct:
            if c in SKIP_SYMBOLS:
                continue

        if accents and unicodedata.combining(c):
            continue

        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        
        n_str.append(c)
        c_prev = c

    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    texto = texto.replace('\n', ' ').replace('\r', ' ')
    return texto


from nltk.corpus import stopwords

def eliminar_stopwords(texto, idioma="spanish"):
    _STOPWORDS = stopwords.words(idioma)
    texto = [t for t in texto.split() if t not in _STOPWORDS]
    return " ".join(texto)


### Tokenización
Obtener las oraciones o tokens del texto 

In [2]:
import nltk

def tokenizar_por_oracion(texto):
    tokenizador_oraciones = nltk.data.load('tokenizers/punkt/spanish.pickle')
    oraciones = tokenizador_oraciones.tokenize(texto)
    return oraciones


from  nltk import word_tokenize
from nltk.stem import SnowballStemmer

def tokenizador_palabra(texto):
    tokens = word_tokenize(texto)
    return tokens

### Stemming - Lematización
reducir las palabras a su raíz

In [3]:
from nltk.stem import SnowballStemmer

def stemming(texto, idioma):
    stemmer = SnowballStemmer(idioma)
    texto = stemmer.stem(texto)
    return texto


import spacy
from spacy.lang.es.examples import sentences 

def lematizar_texto(input_str):
    NLP_ES = spacy.load("es_core_news_md")
    doc = NLP_ES(input_str)
    lemas = (token.lemma_ for token in doc)
    return " ".join(lemas)


import pandas as pd

def lematizar_dataframe(df, columna_texto, n_hilos=4):
    """
    Aplica la lematización a una columna de un DataFrame usando nlp.pipe() 
    para procesamiento paralelo.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        columna_texto (str): Nombre de la columna que contiene el texto.
        n_hilos (int): Número de procesos (hilos) a utilizar.

    Returns:
        pd.Series: Serie con el texto lematizado.
    """
    NLP_ES = spacy.load("es_core_news_md")

    # Obtener el generador de documentos procesados
    docs = NLP_ES.pipe(df[columna_texto], n_process=n_hilos)

    # Convertir los documentos procesados a una lista de lemas
    lemas_list = []
    for doc in docs:
        # Unimos los lemas de cada token en un solo string
        lemas_list.append(" ".join([token.lemma_ for token in doc]))

    return pd.Series(lemas_list, index=df.index)



## Vectorización

### TF-IDF - Embeddings

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords

def vectorizar_tf_idf(df,
                      columna_texto="procesado",
                      stopwords = False,
                      idioma="spanish",
                      max_features=5000):
    """
    df: DataFrame con los textos
    columna_texto: columna donde está el texto preprocesado (string)
    idioma: idioma para las stopwords de NLTK
    max_features: número máximo de términos en el vocabulario
    """

    if stopwords: 
        STOPWORDS = nltk_stopwords.words(idioma)
    else:
        STOPWORDS = None

    tfidf_vectorizer = TfidfVectorizer(
        stop_words=STOPWORDS,
        ngram_range=(1, 1),
        max_features=max_features
    )

    # Usamos la columna ya procesada (normalizada, lematizada, sin stopwords)
    textos = df[columna_texto].values

    # Ajustar y transformar
    tfidf_matriz = tfidf_vectorizer.fit_transform(textos)

    # Nombres de características
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Matriz densa en DataFrame (fila = ejemplo, columna = término)
    tfidf_df = pd.DataFrame(
        tfidf_matriz.toarray(),
        columns=feature_names,
        index=df.index
    )

    # Devolvemos también el vectorizador por si lo necesitas después
    return tfidf_df, tfidf_vectorizer


import fasttext

def vectorizar_embeddings(df, ruta_modelo):
    # Cargar el modelo
    ft = fasttext.load_model(ruta_modelo)

    # Crear los vectores densos para cada texto
    df["embeddings"] = df["procesado"].map(lambda lista_tokens: ft.get_sentence_vector(" ".join(lista_tokens)))

# nuevotf = vectorizar_tf_idf(textos, "spanish")
# nuevoem = vectorizar_embeddings(textos, "./datos/bins/MX.bin")

# Entrenamiento

## Definición de la arquitectura

In [5]:
import torch.nn as nn
import torch.nn.init as init

class MLPDinamico(nn.Module):
    def __init__(self,
                 input_size,
                 num_clases,
                 neuronas_por_capa,
                 activaciones_ocultas,
                 dropout_rates=None,
                 activacion_salida="softmax"):
        super().__init__()

        # Validaciones
        assert len(activaciones_ocultas) == len(neuronas_por_capa), "Todos los arrays deben tener la misma longitud"

        self.input_size = input_size
        self.output_size = num_clases

        # Mapeo de funciones de activación
        self.activacion_diccionario = {
            'relu': nn.ReLU(),
            'sigmoid': nn.Sigmoid(),
            'tanh': nn.Tanh(),
            'leaky_relu': nn.LeakyReLU(),
            'selu': nn.SELU(),
            ## Clasificaión
            'softmax': nn.Softmax(dim=1),
            'log_softmax': nn.LogSoftmax(dim=1)
        }

        # Dropout
        num_capas_ocultas = len(neuronas_por_capa)
        if dropout_rates is None:
            # asignar dropout en 0, significa que no esta apagando neuronas
            dropout_rates = [0.0] * num_capas_ocultas

        assert len(dropout_rates) == num_capas_ocultas, \
            f"Error en los dropout_rates: esperadas {num_capas_ocultas}, recibidas {len(dropout_rates)}"

        # Construcción dinámica de capas
        capas = []
        tam_entrada_actual = input_size

        for i in range(num_capas_ocultas):
            n_salida = neuronas_por_capa[i]
            activacion_str = activaciones_ocultas[i]
            dropout_rate = dropout_rates[i]

            # Capa lineal
            capas.append(nn.Linear(tam_entrada_actual, n_salida))

            # Funció de activación
            capas.append(self._get_activacion(activacion_str))

            # Dropout
            if dropout_rate > 0:
                capas.append(nn.Dropout(dropout_rate))

            # Pasar a la siguiente capa
            tam_entrada_actual = n_salida

        # Ultima capa oculta
        capas.append(nn.Linear(tam_entrada_actual, self.output_size))

        # Activación de salida
        if activacion_salida:
            capas.append(self._get_activacion(activacion_salida))

        self.model = nn.Sequential(*capas)

    def _get_activacion(self, activacion):
        if activacion.lower() in self.activacion_diccionario:
            return self.activacion_diccionario[activacion.lower()]
        else:
            raise ValueError(f"Funcion de activacion no implementada {activacion}")
        
    def inicializar_pesos(self):
        for m in self.modules():
            # **Asegúrate de que 'm' es una capa Lineal antes de acceder a .weight y .bias**
            if isinstance(m, nn.Linear): 
                init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
        
    def forward(self, X):
        return self.model(X)

## Cargar datos

In [6]:
import pandas as pd

datos = pd.read_json("./datos/dataset_polaridad_es.json", lines=True)

## Procesamiento de datos

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import os

# Normalizar texto
datos["procesado"] = datos["text"].apply(lambda x: normaliza_texto(x, punct=False, accents=False, minusculas=True))

# Lematizar
datos["procesado"] = lematizar_dataframe(df=datos, columna_texto="procesado", n_hilos=16)

# Stopwords
datos["procesado"] = datos["procesado"].apply(eliminar_stopwords)

# Pesado de datos (vectorizar)
# datos["procesado"] = datos["procesado"].apply(tokenizador_palabra)


# ==== Vectorizar ====
#Vectorizar con fastText
# vectorizar_embeddings(datos, "./datos/bins/MX.bin")

# # Crear matriz de datos
# datos["embeddings"].to_numpy()
# datos["embeddings"].to_numpy().shape

# # Matriz de caracteristicas
# X = np.vstack(datos['embeddings'].to_numpy())
# Y = datos["klass"].to_numpy()

# Vectorizar con TF-IDF
print("Vectorizando con TF-IDF...")
tfidf_df, tfidf_vectorizer = vectorizar_tf_idf(
    df=datos,
    stopwords=False,
    columna_texto="procesado",  # usamos el texto ya normalizado/lematizado/sin stopwords
    idioma="spanish",
    max_features=30000
)

print("   TF-IDF listo. Forma de la matriz:", tfidf_df.shape)

Matriz de características (numpy)
X = tfidf_df.to_numpy().astype(np.float32)
Y = datos["klass"].to_numpy()

# Codificar las clases
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)

# Separar evaluaciones
X_train, X_val, Y_train_num, Y_val_num = train_test_split(
    X,
    Y_encoded, 
    test_size=0.1, 
    stratify=Y_encoded, 
    random_state=42
)

print(f"   División completada. Train: {X_train.shape[0]}, Val: {X_val.shape[0]}")

   División completada. Train: 3535, Val: 393


## Convertir a tensores

In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convertir a tensores
X_train_tensor = torch.from_numpy(X_train).float()       # features -> float32
X_val_tensor   = torch.from_numpy(X_val).float()

Y_train_tensor = torch.from_numpy(Y_train_num).long()    # etiquetas -> long (para CrossEntropyLoss)
Y_val_tensor   = torch.from_numpy(Y_val_num).long()


# Crear datasets
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, Y_val_tensor)

# Crear DataLoaders
batch_size = 64  # puedes ajustar esto

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


INPUT_SIZE  = X_train.shape[1]          # dimensión del embedding - TF-IDF
NUM_CLASSES = len(np.unique(Y_encoded)) # o len(le.classes_)


## Guardar a JSON

In [9]:
import json

# --- Función de Ayuda para JSON ---
def append_to_json_file(filename, data):
    """Carga el contenido de un archivo JSON y añade un nuevo registro."""
    try:
        # Cargar los datos existentes
        with open(filename, 'r') as f:
            current_data = json.load(f)
    except FileNotFoundError:
        # Crear la lista si el archivo no existe
        current_data = []
    except json.JSONDecodeError:
        # Manejar archivo vacío o corrupto
        current_data = []

    # Añadir el nuevo dato
    current_data.append(data)

    # Escribir de vuelta al archivo
    with open(filename, 'w') as f:
        json.dump(current_data, f, indent=4)


## Entrenamiento

In [10]:
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def train_one_model(model,
                    train_loader,
                    val_loader,
                    criterion,
                    optimizer_class,
                    optimizer_kwargs,
                    num_epochs,
                    device,
                    ruido=0.0):

    model.to(device)
    model.inicializar_pesos()
    optimizer = optimizer_class(model.parameters(), **optimizer_kwargs)

    best_val_acc = 0.0

    for epoch in range(num_epochs):

        # ------- ENTRENAMIENTO -------
        model.train()
        train_preds = []
        train_labels = []
        running_loss = 0

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            # --- Inyección de ruido solo en TRAIN ---
            if ruido > 0:
                ruido = torch.randn_like(X_batch) * ruido
                X_batch_input = X_batch + ruido
            else:
                X_batch_input = X_batch
            # ----------------------------------------

            optimizer.zero_grad()

            outputs = model(X_batch_input)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * X_batch.size(0)

            preds = outputs.argmax(dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(y_batch.cpu().numpy())

        train_loss = running_loss / len(train_labels)
        train_acc  = accuracy_score(train_labels, train_preds)
        train_prec = precision_score(train_labels, train_preds, average='macro', zero_division=0)
        train_rec  = recall_score(train_labels, train_preds, average='macro', zero_division=0)
        train_f1   = f1_score(train_labels, train_preds, average='macro', zero_division=0)

        # ------- VALIDACIÓN -------
        model.eval()
        val_preds = []
        val_labels = []
        val_loss_sum = 0

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val = X_val.to(device)
                y_val = y_val.to(device)

                outputs = model(X_val)
                loss = criterion(outputs, y_val)
                val_loss_sum += loss.item() * X_val.size(0)

                preds = outputs.argmax(dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(y_val.cpu().numpy())

        val_loss = val_loss_sum / len(val_labels)
        val_acc  = accuracy_score(val_labels, val_preds)
        val_prec = precision_score(val_labels, val_preds, average='macro', zero_division=0)
        val_rec  = recall_score(val_labels, val_preds, average='macro', zero_division=0)
        val_f1   = f1_score(val_labels, val_preds, average='macro', zero_division=0)

        # ------- IMPRESIÓN DE MÉTRICAS -------
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        print(f" Train  -> Loss: {train_loss:.4f} | Acc: {train_acc:.4f} "
              f"| Prec: {train_prec:.4f} | Rec: {train_rec:.4f} | F1: {train_f1:.4f}")

        print(f" Val    -> Loss: {val_loss:.4f} | Acc: {val_acc:.4f} "
              f"| Prec: {val_prec:.4f} | Rec: {val_rec:.4f} | F1: {val_f1:.4f}")

        # ------- Guardar mejor modelo según accuracy -------
        if val_acc > best_val_acc:
            best_val_acc = val_acc

    return best_val_acc


## Evaluación

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluar_modelo(model, data_loader, device):
    """
    Calcula Accuracy, Precision, Recall y F1 sobre un DataLoader.
    """
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average="macro", zero_division=0)
    rec = recall_score(all_labels, all_preds, average="macro", zero_division=0)
    f1  = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return acc, prec, rec, f1


## Función de Grid Search

In [ ]:
from itertools import product
from torch.utils.data import DataLoader  # por si acaso

def generar_combinaciones(param_grid):
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    for vals in product(*values):
        yield dict(zip(keys, vals))


RESULTS_JSON_FILE = "./resultados/resultados_TF-IDF_Unigramas.json"

def grid_search_mlp(input_size,
                    num_clases,
                    train_loader,
                    val_loader,
                    param_grid,
                    num_epochs=20,
                    device="cpu",
                    optimizer_class=optim.Adam,
                    criterion=None,
                    activacion_salida=None,
                    ruido=0.2):

    if criterion is None:
        criterion = nn.CrossEntropyLoss()

    best_score = 0.0
    best_params = None
    best_state_dict = None

    # Usamos los datasets base para recrear DataLoaders con distinto batch_size
    base_train_dataset = train_loader.dataset
    base_val_dataset   = val_loader.dataset

    for combo in generar_combinaciones(param_grid):
        print("\nProbando combinación de hiperparámetros:")
        for k, v in combo.items():
            print(f"  {k}: {v}")

        # ---- Arquitectura de la MLP ----
        config_capas = combo["config_capas"]
        neuronas_por_capa    = config_capas["neuronas_por_capa"]
        activaciones_ocultas = config_capas["activaciones_ocultas"]
        dropout_rates        = config_capas.get("dropout_rates", None)

        # ---- Hiperparámetros del optimizador ----
        # OJO: el param_grid usa "learning_rate"
        lr           = combo.get("learning_rate", 1e-3)
        weight_decay = combo.get("weight_decay", 0.0)
        beta1        = combo.get("beta1", 0.9)

        # ---- Batch size desde el grid ----
        # Si no está en combo, usamos el batch_size del loader original o 64
        batch_size = combo.get("batch_size",
                               getattr(train_loader, "batch_size", 64))

        # Crear nuevos DataLoaders con este batch_size
        train_loader_combo = DataLoader(base_train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)
        val_loader_combo   = DataLoader(base_val_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

        # ---- Crear el modelo ----
        model = MLPDinamico(
            input_size=input_size,
            num_clases=num_clases,
            neuronas_por_capa=neuronas_por_capa,
            activaciones_ocultas=activaciones_ocultas,
            dropout_rates=dropout_rates,
            activacion_salida=activacion_salida
        )

        optimizer_kwargs = {
            "lr": lr,
            "weight_decay": weight_decay,
            "betas": (beta1, 0.999)   # usamos beta1 del grid
        }

        # ---- Entrenar el modelo (solo entrena, ignoramos su retorno) ----
        train_one_model(
            model=model,
            train_loader=train_loader_combo,
            val_loader=val_loader_combo,
            criterion=criterion,
            optimizer_class=optimizer_class,
            optimizer_kwargs=optimizer_kwargs,
            num_epochs=num_epochs,
            device=device
        )

        # ---- Evaluar métricas completas en validación ----
        val_acc, val_prec, val_rec, val_f1 = evaluar_modelo(
            model,
            val_loader_combo,
            device
        )

        print(
            f"Métricas validación -> "
            f"Accuracy: {val_acc:.4f}, "
            f"Precision: {val_prec:.4f}, "
            f"Recall: {val_rec:.4f}, "
            f"F1: {val_f1:.4f}"
        )

        # ---- Guardar resultado en JSON ----
        json_record = {
            "params": {
                "config_capas": config_capas,
                "learning_rate": float(lr),
                "weight_decay": float(weight_decay),
                "batch_size": int(batch_size),
                "beta1": float(beta1)
            },
            "val_accuracy": float(val_acc),
            "val_precision": float(val_prec),
            "val_recall": float(val_rec),
            "val_f1": float(val_f1)
        }

        append_to_json_file(RESULTS_JSON_FILE, json_record)

        # ---- Actualizar mejor modelo según accuracy de validación ----
        if val_acc > best_score:
            best_score = val_acc
            best_params = combo
            best_state_dict = model.state_dict()

    print("\n===== RESULTADOS GRID SEARCH =====")
    print(f"Mejor accuracy de validación: {best_score:.4f}")
    print("Mejores parámetros:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")

    return best_params, best_score, best_state_dict

## Ejemplo

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Mover el modelo a la GPU si está disponible
if torch.cuda.is_available():
    device = "cuda"
elif torch.mps.is_available():
    device = "mps"
else:
    device = "cpu"

param_grid = {
    # Diferentes arquitecturas de la MLP
    "config_capas": [
        {
            "neuronas_por_capa": [128, 64],
            "activaciones_ocultas": ["relu", "relu"],
            "dropout_rates": [0.3, 0.3]
        },
        # Modelo mediano
        {
            "neuronas_por_capa": [256, 128, 64],
            "activaciones_ocultas": ["relu", "relu", "relu"],
            "dropout_rates": [0.5, 0.5, 0.5]
        },
        # Más neuronas, dropout moderado
        {
            "neuronas_por_capa": [256, 256, 128],
            "activaciones_ocultas": ["relu", "relu", "relu"],
            "dropout_rates": [0.4, 0.4, 0.4]
        },
        # Modelo grande con bastante regularización
        {
            "neuronas_por_capa": [512, 256, 128],
            "activaciones_ocultas": ["relu", "relu", "relu"],
            "dropout_rates": [0.5, 0.5, 0.5]
        },
    ],

        "optimizer_kwargs": [
        {"lr": 1e-3, "weight_decay": 1e-4},
        {"lr": 3e-4, "weight_decay": 1e-4},
        {"lr": 1e-4, "weight_decay": 1e-5}
    ],

    # Tasas de aprendizaje a probar
    "learning_rate": [1e-4, 3e-4, 1e-3],

    # Regularización L2 (weight decay)
    "weight_decay": [0.0, 1e-4, 1e-3],

    # Batch size
    "batch_size": [128, 64, 32, 16],

    # Opcional: momento de Adam (beta1)
    "beta1": [0.9, 0.95]
}


best_params, best_score, best_state_dict = grid_search_mlp(
    input_size=INPUT_SIZE,
    num_clases=NUM_CLASSES,
    train_loader=train_loader,
    val_loader=val_loader,
    param_grid=param_grid,
    num_epochs=20,
    device=device,
    optimizer_class=optim.Adam,
    criterion=nn.CrossEntropyLoss(),
    activacion_salida=None,   # MUY IMPORTANTE si uso CrossEntropyLoss
    ruido=0.0 # Noise Injection
)


Probando combinación de hiperparámetros:
  config_capas: {'neuronas_por_capa': [128, 64], 'activaciones_ocultas': ['relu', 'relu'], 'dropout_rates': [0.3, 0.3]}
  learning_rate: 0.0001
  weight_decay: 0.0
  batch_size: 128
  beta1: 0.9

Epoch 1/100
 Train  -> Loss: 1.0913 | Acc: 0.4147 | Prec: 0.3226 | Rec: 0.3279 | F1: 0.3034
 Val    -> Loss: 1.0813 | Acc: 0.4733 | Prec: 0.1578 | Rec: 0.3333 | F1: 0.2142

Epoch 2/100
 Train  -> Loss: 1.0753 | Acc: 0.4710 | Prec: 0.2817 | Rec: 0.3354 | F1: 0.2293
 Val    -> Loss: 1.0671 | Acc: 0.4733 | Prec: 0.1578 | Rec: 0.3333 | F1: 0.2142

Epoch 3/100
 Train  -> Loss: 1.0628 | Acc: 0.4727 | Prec: 0.2828 | Rec: 0.3345 | F1: 0.2192
 Val    -> Loss: 1.0564 | Acc: 0.4733 | Prec: 0.1578 | Rec: 0.3333 | F1: 0.2142

Epoch 4/100
 Train  -> Loss: 1.0548 | Acc: 0.4721 | Prec: 0.1575 | Rec: 0.3331 | F1: 0.2139
 Val    -> Loss: 1.0508 | Acc: 0.4733 | Prec: 0.1578 | Rec: 0.3333 | F1: 0.2142

Epoch 5/100
 Train  -> Loss: 1.0521 | Acc: 0.4724 | Prec: 0.1575 | Rec

KeyboardInterrupt: 